# 🚦 SRIG — Civic Score Dataset Generator
### Smart Road India Grid | Model 3 — TabTransformer
**Yash Jani | Red and White Institute, Surat**

---
Is notebook mein hum **10,000 drivers ka synthetic dataset** banayenge jo SRIG ke Civic Score system ko train karne ke liye use hoga.

## 📦 Step 1 — Libraries Import Karo

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

print('✅ Libraries successfully import ho gayi!')

## 🎲 Step 2 — Raw Features Generate Karo

Hum 10,000 drivers ke liye realistic data banayenge jaise:
- Helmet compliance
- Signal violations
- Speed violations
- School zone violations
- aur bahut kuch...

In [ ]:
np.random.seed(42)
NUM_DRIVERS = 10000

driver_ids             = [f'DL-{str(i).zfill(5)}' for i in range(1, NUM_DRIVERS + 1)]
helmet_compliance      = np.random.randint(0, 31, NUM_DRIVERS)
signal_violations      = np.random.randint(0, 20, NUM_DRIVERS)
speed_violations       = np.random.randint(0, 15, NUM_DRIVERS)
school_zone_violations = np.random.randint(0, 10, NUM_DRIVERS)
drunk_driving_cases    = np.random.randint(0, 3,  NUM_DRIVERS)
accident_history       = np.random.randint(0, 5,  NUM_DRIVERS)
years_driving          = np.random.randint(1, 40, NUM_DRIVERS)
training_completed     = np.random.randint(0, 2,  NUM_DRIVERS)
complaints_filed       = np.random.randint(0, 10, NUM_DRIVERS)
city_tier              = np.random.choice([1, 2, 3], NUM_DRIVERS)

print(f'✅ {NUM_DRIVERS} drivers ka raw data generate ho gaya!')

## 🧮 Step 3 — Civic Score Calculate Karo

SRIG ke rules ke according score calculate hoga:
- ✅ Helmet compliance → **+10 points/day**
- ❌ Signal violation → **-30 points**
- ❌ School zone violation → **-40 points**
- ❌ Drunk driving → **-100 points**

In [ ]:
score = np.full(NUM_DRIVERS, 500.0)  # Base score 500

score += helmet_compliance      * 10
score -= signal_violations      * 30
score -= speed_violations       * 25
score -= school_zone_violations * 40
score -= drunk_driving_cases    * 100
score -= accident_history       * 50
score += years_driving          * 5
score += training_completed     * 50
score -= complaints_filed       * 20
score -= (city_tier - 1)        * 30

# 0 se 1000 ke beech clip karo
score = np.clip(score, 0, 1000)

print(f'✅ Civic Score calculate ho gaya!')
print(f'   Min Score : {score.min():.0f}')
print(f'   Max Score : {score.max():.0f}')
print(f'   Avg Score : {score.mean():.0f}')

## 🏅 Step 4 — Risk Level / Tier Assign Karo

| Score Range | Tier |
|---|---|
| 900–1000 | 🥇 PLATINUM |
| 750–899 | 🥈 GOLD |
| 600–749 | 🥉 SILVER |
| 400–599 | 🔴 BRONZE |
| 0–399 | ⚠️ HIGH_RISK |

In [ ]:
def assign_risk(s):
    if s >= 900:   return 'PLATINUM'
    elif s >= 750: return 'GOLD'
    elif s >= 600: return 'SILVER'
    elif s >= 400: return 'BRONZE'
    else:          return 'HIGH_RISK'

risk_labels = [assign_risk(s) for s in score]

print('✅ Risk levels assign ho gaye!')

## 📋 Step 5 — DataFrame Banao

In [ ]:
df = pd.DataFrame({
    'driver_id':              driver_ids,
    'helmet_compliance_days': helmet_compliance,
    'signal_violations':      signal_violations,
    'speed_violations':       speed_violations,
    'school_zone_violations': school_zone_violations,
    'drunk_driving_cases':    drunk_driving_cases,
    'accident_history':       accident_history,
    'years_driving':          years_driving,
    'training_completed':     training_completed,
    'complaints_filed':       complaints_filed,
    'city_tier':              city_tier,
    'civic_score':            score.round(2),
    'risk_level':             risk_labels
})

print(f'✅ DataFrame ready!')
df.head(10)

## 📊 Step 6 — Data Visualize Karo

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('SRIG — Civic Score Dataset Analysis', fontsize=16, fontweight='bold')

# Plot 1: Risk Level Distribution
risk_counts = df['risk_level'].value_counts()
colors = ['#FFD700', '#C0C0C0', '#CD7F32', '#FF4444', '#4444FF']
axes[0].bar(risk_counts.index, risk_counts.values, color=colors[:len(risk_counts)])
axes[0].set_title('Risk Level Distribution')
axes[0].set_xlabel('Risk Level')
axes[0].set_ylabel('Number of Drivers')
axes[0].tick_params(axis='x', rotation=15)

# Plot 2: Civic Score Distribution
axes[1].hist(df['civic_score'], bins=50, color='steelblue', edgecolor='white')
axes[1].set_title('Civic Score Distribution')
axes[1].set_xlabel('Civic Score (0-1000)')
axes[1].set_ylabel('Number of Drivers')
axes[1].axvline(df['civic_score'].mean(), color='red', linestyle='--', label=f'Mean: {df["civic_score"].mean():.0f}')
axes[1].legend()

# Plot 3: Helmet Compliance vs Score
axes[2].scatter(df['helmet_compliance_days'], df['civic_score'], alpha=0.1, color='green', s=5)
axes[2].set_title('Helmet Compliance vs Civic Score')
axes[2].set_xlabel('Helmet Compliance Days')
axes[2].set_ylabel('Civic Score')

plt.tight_layout()
plt.savefig('data/dataset_analysis.png', dpi=150, bbox_inches='tight')
plt.show()
print('✅ Charts save ho gaye: data/dataset_analysis.png')

## 📈 Step 7 — Dataset Stats Dekho

In [ ]:
print('📊 Risk Level Distribution:')
print(df['risk_level'].value_counts())
print('\n📈 Civic Score Statistics:')
print(df['civic_score'].describe().round(2))
print('\n🔢 Feature Statistics:')
df.drop(columns=['driver_id', 'risk_level']).describe().round(2)

## 💾 Step 8 — CSV File Save Karo

In [ ]:
os.makedirs('data', exist_ok=True)
df.to_csv('data/civic_score_dataset.csv', index=False)

print('🎉 Dataset successfully save ho gaya!')
print(f'   📁 Location : data/civic_score_dataset.csv')
print(f'   📊 Total Rows : {len(df):,}')
print(f'   📋 Total Columns : {len(df.columns)}')
print('\n✅ Ab train_model.py / train_model.ipynb run karo!')